# 非循环依赖访问者模式

## 传统的访问者模式（visitor-pattern）

![visitor-patter](pic/visitor-pattern.jpg)

> - Element是被访问的对象接口，A和B实现了Element的接口，是实际的被访问对象，通过Element的封装接受（Accept）访问者
> - Vistor是访问者的对象接口，对于A和B分别定义了访问的行为visitA和visitB，MyFunctionVisitor是实际的访问者，实现Visitor的方法
> - 由上图可见，A和B依赖于Element，Element依赖于Visitor，Visitor却又依赖于A和B（因为需要分别实现访问行为），这是一个循环依赖
> - Visitor模式解决了当添加处理方法时不用改动数据的结构定义，如当我们需要一种新的处理A和B数据的方式时，我们只需要添加一个新的Visitor实现类即可，A和B将不会受到影响
> - 但实现了Visitor之后，如果我们需要添加一个新的受访者数据结构时，由于三者的循环依赖，Visitor接口不可避免的需要改变，同时导致所有已经实现Visitor接口的类也需要跟着改变

## 非循环依赖访问者

![ORM](pic/acyclic-visitor-pattern.jpg)

### 目标

为当前的受访者数据模型添加一个新的数据结构时，不需要修改现有的访问继承结构和代码

### 应用场景

- 当需要动态增加受访者数据结构或需要动态增加访问方法时
- 当对不同的受访者需要采用差别比较大的的访问方法时
- 当受访者经常会增加基于Element的数据模型时
- 当重新编译、重新发布基于Element的数据模型变得非常昂贵甚至不可能时

In [1]:
// 退化的Visitor接口，空定义
public interface DeviceVisitor {}

// Element基类
public abstract class Device {
    String deviceName;
    public abstract void accept(DeviceVisitor visitor) throws IllegalStateException;
    public String getDeviceName() { return deviceName; }
}

In [2]:
// 实际的Visitor接口，根据不同的Element数据接口进行定义
public interface MonitorVisitor extends DeviceVisitor {
    void visitMonitor(Device monitor);
}
public interface CpuVisitor extends DeviceVisitor {
    void visitCpu(Device cpu);
}
public interface MemoryVisitor extends DeviceVisitor {
    void visitMemory(Device memory);
}

In [3]:
// 一些受访者实现类
public class Monitor extends Device {
    private String brand;
    public Monitor(String brand) { 
        deviceName = "显示器";
        this.brand = brand;
    }
    public String getBrand() { return brand; }
    
    @Override
    public void accept(DeviceVisitor visitor) throws IllegalStateException {
        if(visitor instanceof MonitorVisitor) {
            ((MonitorVisitor) visitor).visitMonitor(this);
        } else {
            throw new IllegalStateException("非显示器访问者无法访问显示器对象");
        }
    }
    
    @Override
    public String toString() {
        return String.format("%s:\n\t品牌: %s", deviceName, brand);
    }
}

public class Cpu extends Device {
    private int cores;
    public Cpu(int cores) {
        deviceName = "处理器";
        this.cores = cores;
    }
    public int getCores() { return cores; }
    
    @Override
    public void accept(DeviceVisitor visitor) throws IllegalStateException {
        if(visitor instanceof CpuVisitor) {
            ((CpuVisitor) visitor).visitCpu(this);
        } else {
            throw new IllegalStateException("非处理器访问者无法访问处理器对象");
        }
    }
    
    @Override
    public String toString() {
        return String.format("%s:\n\t核心数: %d", deviceName, cores);
    }
}

public class Memory extends Device {
    private int capacity;
    public Memory(int capacity) {
        deviceName = "内存";
        this.capacity = capacity;
    }
    public int getCapacity() { return capacity; }
    
    @Override
    public void accept(DeviceVisitor visitor) throws IllegalStateException {
        if(visitor instanceof MemoryVisitor) {
            ((MemoryVisitor) visitor).visitMemory(this);
        } else {
            throw new IllegalStateException("非内存访问者无法访问内存对象");
        }
    }
    
    @Override
    public String toString() {
        return String.format("%s:\n\t容量: %dGiB", deviceName, capacity);
    }
}

In [4]:
// 一些访问者类
// 硬盘访问者，只能访问CPU和内存
public class HardDriveVisitor implements CpuVisitor, MemoryVisitor {
    @Override
    public void visitCpu(Device cpu) {
        System.out.println("硬盘正在访问 --> " + cpu);
    }
    
    @Override
    public void visitMemory(Device memory) {
        System.out.println("硬盘正在访问 --> " + memory);
    }
}

// 显卡访问者，可以访问CPU、内存及显示器
public class VideoCardVisitor implements CpuVisitor, MemoryVisitor, MonitorVisitor {
    @Override
    public void visitCpu(Device cpu) {
        System.out.println("显卡正在访问 --> " + cpu);
    }
    
    @Override
    public void visitMemory(Device memory) {
        System.out.println("显卡正在访问 --> " + memory);
    }
    
    @Override
    public void visitMonitor(Device monitor) {
        System.out.println("显卡正在访问 --> " + monitor);
    }
}

In [5]:
// 测试主类
public class App {
    public static void main(String[] args) {
        Monitor monitor = new Monitor("DELL");
        Cpu cpu = new Cpu(8);
        Memory memory = new Memory(32);
        HardDriveVisitor hdVisitor = new HardDriveVisitor();
        VideoCardVisitor vcVisitor = new VideoCardVisitor();
        cpu.accept(hdVisitor);
        memory.accept(hdVisitor);
        cpu.accept(vcVisitor);
        memory.accept(vcVisitor);
        monitor.accept(vcVisitor);
        monitor.accept(hdVisitor);
    }
}

In [6]:
new App().main(null)

硬盘正在访问 --> 处理器:
	核心数: 8
硬盘正在访问 --> 内存:
	容量: 32GiB
显卡正在访问 --> 处理器:
	核心数: 8
显卡正在访问 --> 内存:
	容量: 32GiB
显卡正在访问 --> 显示器:
	品牌: DELL


EvalException: 非显示器访问者无法访问显示器对象